# Az intercepció hatása a növényzet számára hasznosítható csapadék mértékének alakulására

Az erdei fák számára felvehető csapadék mértékét jelentősen befolyásolja az a vízmennyiség, amit a lombfelület megköt és újra elpárologtat anélkül, hogy az elérhetné a talajfelszínt. Hazai környezetben fenyő esetén a talajfelszínen található vastag száraz tűavar ezt a hatás jelentősen fokozhatja. Az élő lomb és a száraz avar együttes hatásaként visszatartott csapadékmennyiség elérheti az 5-10 mm mértéket, ami sok esetben a teljes csapadékesemény 30-50%-át is jelentheti.

<img src="https://github.com/jsheperd/forest_water/blob/master/img/csapadek.png?raw=true" width="300" height="300" />

### Hipotézis

A növényzet számára felvehető csapadék alakulását nem csak az éves csapadékösszeg befolyásolja, de az eloszlási mintázat is meghatározó. Figyelembe kell venni azt, hogy az év adott szakaszában mekkora a lombfelület nagysága, valamint azt, hogy az egyes csapadékesemények mértékének az eloszlása hogyan alakul. A felszíni elfolyást és a mélybeszivárgást első vizsgáatainkben nem vesszük figyelembe.

### Felhasznált adatok

Az [Országos meteorológia Szolgálat 1901-2010 éghajalati adasorai](http://www.met.hu/eghajlat/magyarorszag_eghajlata/eghajlati_adatsorok/) napi csapdékeseményeket is tartalmaznak, ami alkalmas arra, hogy becsléseket tegyünk az aktuális csapadékesemény intercepciós veszteségére vonatkozóan 110 éves időszakra vonatkozóan.

In [2]:
import pandas as pd
from pandas import Series

szo_daily = pd.read_csv('https://raw.githubusercontent.com/jsheperd/forest_water/master/data/SZO_d.txt', sep=';')
szo_daily['datum'] = pd.to_datetime(szo_daily['#datum'], format='%Y-%m-%d')

In [ ]:
import matplotlib.pyplot as plt

dates = pd.date_range('1901-01-01', '2010-12-31', freq='d')
y = szo_daily["d_rs"]

plt.plot(dates, y)